#### QGIS -> demand 
노선별로 경로 생성
distance 데이터 넣기

In [8]:
import pandas as pd
from collections import defaultdict, deque

# ──────────────────────────────
# 1. 파일 경로
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
xlsx_path = base_path + r"\qgis_export.xlsx"
sheet_name = "LINE"
# ──────────────────────────────

df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
df = df.dropna(subset=["from_node_", "to_node_id"])
df["from_node_"] = df["from_node_"].astype(int)
df["to_node_id"] = df["to_node_id"].astype(int)

print("📌 노선별 단일 경로 (방향 무시)")
for line, group in df.groupby("RLWAY_NM"):
    # 양방향 그래프 생성
    graph = defaultdict(list)
    for u, v in zip(group["from_node_"], group["to_node_id"]):
        graph[u].append(v)
        graph[v].append(u)

    # 노드 연결 상태 진단
    degree = {n: len(adj) for n, adj in graph.items()}
    endpoints = [n for n, d in degree.items() if d == 1]
    if len(endpoints) != 2:
        print(f"⚠️ {line}: 선형 경로 아님 (끝 노드 {len(endpoints)}개)")
        continue

    # 한쪽 끝에서부터 BFS로 경로 복원
    start = endpoints[0]
    visited = set()
    path = []

    def dfs(u):
        visited.add(u)
        path.append(u)
        for v in graph[u]:
            if v not in visited:
                dfs(v)

    dfs(start)
    print(f"{line}: {path}")


📌 노선별 단일 경로 (방향 무시)
KTX강릉선: [22, 23, 34, 51]
경부고속철도: [4, 3, 1, 2, 13, 18, 28, 42, 60, 72, 75, 79]
경부선: [79, 77, 76, 73, 70, 63, 60, 59, 46, 41, 28, 17, 14, 9, 7, 1, 4]
경북선: [41, 38, 36, 40, 43]
경전선: [48, 57, 64, 65, 62, 66, 71, 73]
동해선: [55, 58, 69, 74, 72]
영동선: [51, 55, 54, 50, 43]
장항선: [14, 11, 10, 15, 16, 27, 30]
전라선: [68, 62, 56, 47, 45, 33, 30]
중앙선: [3, 5, 6, 8, 12, 22, 29, 31, 35, 39, 43, 49, 52, 67, 72, 78, 79]
충북선: [17, 18, 19, 20, 26, 29]
호남고속철도: [48, 37, 30, 21, 18, 13, 2, 1, 3]
호남선: [61, 53, 48, 44, 37, 32, 30, 24, 25, 28]


#### EDGE timestep 데이터 만들기 위해
- distance로 이동시간
- 이동시간 기반 time step 계산
- EDGE 데이터 .json 파일로 저장

In [1]:
import pandas as pd

# ────────────── 설정 ──────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = base_path + r"\qgis_export.xlsx"
output_file = base_path + r"\qgis_export_with_time.xlsx"
sheet_name  = "EDGE"
# ────────────────────────────────

# 1. 원본 엑셀 전체 읽기 (모든 시트 보존 목적)
xlsx_all = pd.read_excel(input_file, sheet_name=None)

# 2. EDGE 시트만 수정
df_edge = xlsx_all[sheet_name]

# 3. time_step 계산
if 'distance' not in df_edge.columns:
    raise ValueError("'distance' 열이 EDGE 시트에 없습니다.")
df_edge['time_step'] = df_edge['distance'] / 150 * 60

# 4. 수정한 EDGE 시트 다시 넣기
xlsx_all[sheet_name] = df_edge

# 5. 전체 시트를 새 파일로 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet, df in xlsx_all.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

print(f"✅ 완료: time_step 열이 추가된 파일이 저장됨 →\n{output_file}")


✅ 완료: time_step 열이 추가된 파일이 저장됨 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_time.xlsx


In [2]:
import pandas as pd
import math

# ────────────── 설정 ──────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = base_path + r"\qgis_export_with_time.xlsx"
output_file = base_path + r"\qgis_export_with_step.xlsx"
sheet_name  = "EDGE"
# ────────────────────────────────

# 1. 모든 시트 읽기
xlsx_all = pd.read_excel(input_file, sheet_name=None)

# 2. EDGE 시트 수정
df_edge = xlsx_all[sheet_name]

# 3. time_step 계산 (5로 나누고 올림)
if 'time(min)' not in df_edge.columns:
    raise ValueError("'time(min)' 열이 EDGE 시트에 없습니다.")

df_edge['time_step'] = df_edge['time(min)'].apply(lambda x: math.ceil(x / 5))

# 4. 덮어쓰기
xlsx_all[sheet_name] = df_edge

# 5. 새 파일로 저장
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for sheet, df in xlsx_all.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

print(f"✅ 완료: 'time_step' 계산 후 저장됨 →\n{output_file}")


✅ 완료: 'time_step' 계산 후 저장됨 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_step.xlsx


In [5]:
import pandas as pd
import json
import os
from collections import OrderedDict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = os.path.join(base_path, "qgis_export_with_step.xlsx")
sheet_name  = "EDGE"
output_json = os.path.join(base_path, "edges.json")
# ───────────────────────────────────

# 1. 데이터 불러오기
df = pd.read_excel(input_file, sheet_name=sheet_name)

# 2. 필요한 열 확인
required_cols = ['edge_id', 'from_node_id', 'to_node_id', 'time_step']
missing = set(required_cols) - set(df.columns)
if missing:
    raise ValueError(f"다음 열이 누락되었습니다: {missing}")

# 3. 정수형 edge_id 기준으로 정렬
df = df.dropna(subset=['edge_id'])
df['edge_id'] = df['edge_id'].astype(int)
df = df.sort_values(by='edge_id')

# 4. edge 딕셔너리 생성 (OrderedDict로 순서 유지)
edges = OrderedDict()

for _, row in df.iterrows():
    eid = f"e{int(row['edge_id'])}"
    n_from = f"n{int(row['from_node_id'])}"
    n_to   = f"n{int(row['to_node_id'])}"
    tstep  = int(row['time_step'])

    edges[eid] = (n_from, n_to, tstep)

# 5. JSON 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(edges, f, indent=4)

print(f"✅ 정렬된 edges.json 저장 완료 →\n{output_json}")


✅ 정렬된 edges.json 저장 완료 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\edges.json


In [6]:
import pandas as pd
import os
import json

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
xlsx_file = os.path.join(base_path, "qgis_export_with_step.xlsx")
sheet_name = "TRAIN"
output_json = os.path.join(base_path, "routes_nodes.json")
# ───────────────────────────────────

# 1. TRAIN 시트 읽기
df = pd.read_excel(xlsx_file, sheet_name=sheet_name)

# 2. routes_nodes 딕셔너리 생성
routes_nodes = {}

for _, row in df.iterrows():
    name = str(row["RLWAY_NM"]).strip()
    nodes_str = str(row["NODE"]).strip()

    # 노드 문자열 파싱 → ['n4', 'n3', 'n1', ...]
    node_list = [f"n{int(n)}" for n in nodes_str.split("-") if n.isdigit()]

    # 정방향 및 역방향 저장
    routes_nodes[f"{name}1"] = node_list
    routes_nodes[f"{name}2"] = node_list[::-1]

# 3. JSON 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(routes_nodes, f, indent=4, ensure_ascii=False)

print(f"✅ 저장 완료: {output_json}")


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\routes_nodes.json


In [7]:
import json
import os
from collections import OrderedDict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_json = os.path.join(base_path, "edges.json")
output_json = os.path.join(base_path, "edges_bidirectional.json")
# ───────────────────────────────────

# 1. 기존 edges.json 불러오기
with open(input_json, "r", encoding="utf-8") as f:
    edges = json.load(f)

# 2. 양방향 딕셔너리 생성 (순서 보존)
edges_bidir = OrderedDict()

for eid, (n1, n2, t) in edges.items():
    # 정방향 edge 추가
    edges_bidir[eid] = [n1, n2, t]
    
    # 역방향 edge 추가
    eid_r = f"{eid}r"
    edges_bidir[eid_r] = [n2, n1, t]

# 3. 새 JSON 파일로 저장
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(edges_bidir, f, indent=4)

print(f"✅ 양방향 edges 저장 완료: {output_json}")


✅ 양방향 edges 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\edges_bidirectional.json


In [8]:
import json
import os
from collections import defaultdict

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
routes_json = os.path.join(base_path, "routes_nodes.json")
output_demand_json = os.path.join(base_path, "demand_template.json")
# ───────────────────────────────────

# 1. routes_nodes.json 불러오기
with open(routes_json, "r", encoding="utf-8") as f:
    routes_nodes = json.load(f)

# 2. demand 생성
demand = {}

for tr, path in routes_nodes.items():
    od_list = []
    for i in range(len(path)-1):
        for j in range(i+1, len(path)):
            o, d = path[i], path[j]
            od_list.append((o, d, None))  # 수요는 아직 없음
    demand[tr] = od_list

# 3. 저장
with open(output_demand_json, "w", encoding="utf-8") as f:
    json.dump(demand, f, indent=4, ensure_ascii=False)

print(f"✅ demand_template.json 저장 완료 →\n{output_demand_json}")


✅ demand_template.json 저장 완료 →
D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_template.json


In [10]:
import json
import os

# ─────────────── 설정 ───────────────
base_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full"
input_file  = os.path.join(base_path, "demand_template.json")
output_file = os.path.join(base_path, "demand_template_compact.json")
# ───────────────────────────────────

# 1. 불러오기
with open(input_file, "r", encoding="utf-8") as f:
    demand = json.load(f)

# 2. 저장 (수동 문자열 포맷)
with open(output_file, "w", encoding="utf-8") as f:
    f.write("{\n")
    for i, (k, od_list) in enumerate(demand.items()):
        f.write(f'    "{k}": [\n')
        line = "        "  # 들여쓰기 시작
        for j, od in enumerate(od_list):
            # JSON 값으로 인코딩 (ensure null/quote 등 맞춤)
            od_json = json.dumps(od, ensure_ascii=False)
            line += od_json
            if j < len(od_list) - 1:
                line += ", "
            if len(line) > 120:
                f.write(line + "\n")
                line = "        "
        if line.strip():
            f.write(line + "\n")
        f.write("    ]")
        if i < len(demand) - 1:
            f.write(",\n")
        else:
            f.write("\n")
    f.write("}\n")

print(f"✅ 성공적으로 저장됨 → {output_file}")


✅ 성공적으로 저장됨 → D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_template_compact.json


#### 역사별 승하차데이터로 임시 journeys demand data 만들기

In [17]:
import os
import pandas as pd

# === 경로 설정 ===
BASE_DIR = r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\data\od-matrix"
in_xlsx  = os.path.join(BASE_DIR, "network-od-matrix-with-journeys.xlsx")
out_xlsx = os.path.join(BASE_DIR, "network-od-matrix-with-journeys-fl.xlsx")

# === 1) 엑셀 읽기 ===
df = pd.read_excel(in_xlsx)

# === 2) node_path 처리 ===
def get_first_last(path_str):
    if pd.isna(path_str):
        return None, None
    parts = [p.strip() for p in str(path_str).split(",") if p.strip() != ""]
    if not parts:
        return None, None
    if len(parts) == 1:
        return parts[0], parts[0]
    return parts[0], parts[-1]

df[["start_node", "end_node"]] = df["node_path"].apply(
    lambda x: pd.Series(get_first_last(x))
)

# === 3) 저장 (원본 열 + 추가 열 같이) ===
df.to_excel(out_xlsx, index=False)

print(f"저장 완료: {out_xlsx}")


저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\data\od-matrix\network-od-matrix-with-journeys-fl.xlsx


In [18]:
import os
import pandas as pd

# === 경로 설정 ===
BASE_DIR = r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK"
od_xlsx   = os.path.join(BASE_DIR, "data", "od-matrix", "network-od-matrix-with-journeys-fl.xlsx")
node_xlsx = os.path.join(BASE_DIR, "json", "qgis_export_uk.xlsx")
out_xlsx  = os.path.join(BASE_DIR, "data", "od-matrix", "network-od-matrix-with-journeys-fl-filtered.xlsx")

# === 1) 데이터 읽기 ===
df = pd.read_excel(od_xlsx)
nodes = pd.read_excel(node_xlsx)

# === 2) wgs84_node_id 집합 ===
valid_nodes = set(nodes["wgs84_node_id"].astype(str))  # 문자열 비교를 안전하게

# === 3) start_node / end_node 값 비교 후 필터링 ===
# start_node, end_node도 문자열로 변환
df["start_node"] = df["start_node"].astype(str)
df["end_node"]   = df["end_node"].astype(str)

filtered = df[
    (df["start_node"].isin(valid_nodes)) | (df["end_node"].isin(valid_nodes))
]

# === 4) 저장 ===
filtered.to_excel(out_xlsx, index=False)

print(f"필터링 완료: {len(filtered)}개 행 남음 → {out_xlsx}")


필터링 완료: 105813개 행 남음 → D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\data\od-matrix\network-od-matrix-with-journeys-fl-filtered.xlsx


In [19]:
import os
import pandas as pd

# === 경로 설정 ===
BASE_DIR   = r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK"
od_xlsx    = os.path.join(BASE_DIR, "data", "od-matrix", "network-od-matrix-with-journeys-fl-filtered.xlsx")
node_xlsx  = os.path.join(BASE_DIR, "json", "qgis_export_uk.xlsx")
out_xlsx   = os.path.join(BASE_DIR, "data", "od-matrix", "network-od-matrix-with-journeys-fl-filtered-mapped.xlsx")

# === 1) 데이터 읽기 ===
df = pd.read_excel(od_xlsx)
nodes = pd.read_excel(node_xlsx, sheet_name="NODE")

# === 2) 매핑 딕셔너리 (wgs84_node_id -> node_id) ===
mapping = dict(zip(nodes["wgs84_node_id"].astype(str), nodes["node_id"]))

# === 3) start_node/end_node 변환 ===
df["o_node_id"] = df["start_node"].astype(str).map(mapping)
df["d_node_id"] = df["end_node"].astype(str).map(mapping)

# === 4) 저장 ===
df.to_excel(out_xlsx, index=False)

print(f"매핑 완료: {out_xlsx}")


매핑 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\data\od-matrix\network-od-matrix-with-journeys-fl-filtered-mapped.xlsx


In [28]:
import pandas as pd
from pathlib import Path

# ─────────────── 경로 설정 ───────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
qgis_path = base_dir / "qgis_export_with_step.xlsx"
ridership_path = base_dir / "station_ridership.xlsx"
# ────────────────────────────────────────

# 1. QGIS NODE 데이터에서 RLNODE_NM 추출
qgis_df = pd.read_excel(qgis_path, sheet_name="NODE")
qgis_nodes = qgis_df["RLNODE_NM"].astype(str).str.strip()

# 2. 승하차 데이터에서 RLNODE_NM 추출 (E열)
ridership_df = pd.read_excel(ridership_path)
ridership_nodes = ridership_df["RLNODE_NM"].astype(str).str.strip()

# 3. 비교
matched = sorted(set(qgis_nodes) & set(ridership_nodes))
only_in_qgis = sorted(set(qgis_nodes) - set(ridership_nodes))

# 4. 출력
print("📌 둘 다 포함된 역 (matched):")
for name in matched:
    print(f" - {name}")

print("\n📌 QGIS에는 있으나 승하차 데이터에는 없는 역 (only_in_qgis):")
for name in only_in_qgis:
    print(f" - {name}")


📌 둘 다 포함된 역 (matched):
 - 경산역
 - 계룡역
 - 곡성역
 - 공주역
 - 광명역
 - 광주송정역
 - 광천역
 - 구례구역
 - 구미역
 - 구포역
 - 기장역
 - 김제역
 - 김천구미역
 - 김천역
 - 나주역
 - 남원역
 - 논산역
 - 능주역
 - 단양역
 - 대구역
 - 대전역
 - 대천역
 - 덕소역
 - 동대구역
 - 동해역
 - 마산역
 - 목포역
 - 물금역
 - 밀양역
 - 벌교역
 - 보성역
 - 봉양역
 - 부산역
 - 삼랑진역
 - 상봉역
 - 상주역
 - 서울역
 - 서창역
 - 수원역
 - 순천역
 - 안동역
 - 양평역
 - 여수엑스포역
 - 영덕역
 - 영등포역
 - 영주역
 - 영천역
 - 예산역
 - 예천역
 - 오송역
 - 온양온천역
 - 용산역
 - 울산역
 - 원주역
 - 의성역
 - 익산역
 - 장성역
 - 장항역
 - 전주역
 - 점촌역
 - 정동진역
 - 정읍역
 - 제천역
 - 진주역
 - 천안역
 - 청량리역
 - 청주역
 - 춘양역
 - 충주역
 - 태화강역
 - 평창역
 - 평택역
 - 포항역
 - 풍기역
 - 횡성역

📌 QGIS에는 있으나 승하차 데이터에는 없는 역 (only_in_qgis):
 - 백산역
 - 보천역
 - 삼척역
 - 신경주역
 - 천안아산역(온양온천)


In [33]:
import pandas as pd
from pathlib import Path

# 파일 경로
base_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
file_path = base_path / "qgis_export.xlsx"

# 엑셀 파일 읽기
train_df = pd.read_excel(file_path, sheet_name="TRAIN")
demand_df = pd.read_excel(file_path, sheet_name="DEMAND")

# 노선 이름 → 노드 경로 dict (n 접두어 붙이기)
line_routes = {}
for _, row in train_df.iterrows():
    line_name = row["RLWAY_NM"].strip()
    node_seq = [f"n{n.strip()}" for n in str(row["NODE"]).split("-")]
    line_routes[line_name] = node_seq

# 노선 이름에서 숫자 제거해서 base name으로 매핑
def get_base_line(line):
    return ''.join(filter(lambda x: not x.isdigit(), line)).strip()

# OD에 대해 Route 생성 함수
def extract_route(line, origin, destination):
    base_line = get_base_line(line)
    nodes = line_routes.get(base_line)
    if not nodes:
        return None
    if origin not in nodes or destination not in nodes:
        return None

    try:
        idx_o = nodes.index(origin)
        idx_d = nodes.index(destination)
        # 정방향 또는 역방향 슬라이싱
        if idx_o <= idx_d:
            path = nodes[idx_o:idx_d + 1]
        else:
            path = nodes[idx_o:idx_d - 1:-1]  # 역방향
        return "-".join(path)
    except ValueError:
        return None


# Route 열 추가
demand_df["Route"] = demand_df.apply(
    lambda row: extract_route(row["Line"], row["Origin"], row["Destination"]),
    axis=1
)

# 저장
output_path = base_path / "demand_with_route_filled.xlsx"
demand_df.to_excel(output_path, index=False)
print(f"✅ 저장 완료: {output_path}")


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\demand_with_route_filled.xlsx


In [37]:
import pandas as pd
from pathlib import Path

# ───────────── 경로 설정 ─────────────
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
xlsx_path = base_dir / "qgis_export.xlsx"
output_path = base_dir / "qgis_export_with_node_id.xlsx"
# ─────────────────────────────────────

# 1. NODE 시트 읽기 (참조용)
node_df = pd.read_excel(xlsx_path, sheet_name="NODE")
node_df = node_df[["RLNODE_NM", "node_id"]].dropna()
node_df["RLNODE_NM"] = node_df["RLNODE_NM"].astype(str).str.strip()

# 2. BOARD_ALIGHT 시트에 node_id 매칭
df = pd.read_excel(xlsx_path, sheet_name="BOARD_ALIGHT")
df["RLNODE_NM"] = df["RLNODE_NM"].astype(str).str.strip()

# 매칭 수행
merged_df = df.merge(node_df, on="RLNODE_NM", how="left")

# 3. 저장
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    merged_df.to_excel(writer, sheet_name="BOARD_ALIGHT", index=False)

print(f"✔️ 저장 완료: {output_path}")


✔️ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_with_node_id.xlsx


In [20]:
import os
import pandas as pd

# === 경로 설정 ===
BASE_DIR = r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK"
od_fp    = os.path.join(BASE_DIR, "data", "od-matrix", "network-od-matrix-with-journeys-fl-filtered-mapped.xlsx")
node_fp  = os.path.join(BASE_DIR, "json", "qgis_export_uk.xlsx")

# === 1) 데이터 읽기 ===
od = pd.read_excel(od_fp)                       # o_node_id, d_node_id, journeys 포함
nodes = pd.read_excel(node_fp, sheet_name="NODE")  # node_id 포함

# 안전한 dtype 정리
od["o_node_id"] = pd.to_numeric(od["o_node_id"], errors="coerce")
od["d_node_id"] = pd.to_numeric(od["d_node_id"], errors="coerce")
od["journeys"]  = pd.to_numeric(od["journeys"],  errors="coerce").fillna(0)

# === 2) 집계: o_node_id/d_node_id별 journeys 합 ===
board_sum  = od.dropna(subset=["o_node_id"]).groupby("o_node_id")["journeys"].sum()
alight_sum = od.dropna(subset=["d_node_id"]).groupby("d_node_id")["journeys"].sum()

# === 3) NODE 시트에 매핑 (없으면 0) ===
nodes["board"]  = nodes["node_id"].map(board_sum).fillna(0)
nodes["alight"] = nodes["node_id"].map(alight_sum).fillna(0)

# 정수로 원하면 아래 주석 해제
# nodes["board"]  = nodes["board"].round(0).astype(int)
# nodes["alight"] = nodes["alight"].round(0).astype(int)

# === 4) 저장: NODE 시트만 교체, 다른 시트 보존 ===
try:
    # pandas>=1.4 권장
    with pd.ExcelWriter(node_fp, engine="openpyxl", mode="a", if_sheet_exists="replace") as w:
        nodes.to_excel(w, sheet_name="NODE", index=False)
except TypeError:
    # if_sheet_exists 미지원 pandas 대응: 전체를 새 파일에 써서 교체
    xls = pd.ExcelFile(node_fp)
    sheets = {sn: pd.read_excel(node_fp, sheet_name=sn) for sn in xls.sheet_names}
    sheets["NODE"] = nodes
    with pd.ExcelWriter(node_fp, engine="openpyxl", mode="w") as w:
        for sn, df in sheets.items():
            df.to_excel(w, sheet_name=sn, index=False)

print("완료: NODE 시트에 board/alight 갱신")


완료: NODE 시트에 board/alight 갱신


In [47]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np

INPUT_XLSX = r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\qgis_export_uk.xlsx"
OUTPUT_XLSX = os.path.splitext(INPUT_XLSX)[0] + "_with_journeys.xlsx"

# === helper: 시트 자동 탐지 ===
def find_sheet_with_columns(sheets_dict, required_cols):
    for name, df in sheets_dict.items():
        cols = {c.strip().lower() for c in df.columns}
        if required_cols.issubset(cols):
            return name
    return None

def find_col(df, target):
    t = target.strip().lower()
    for c in df.columns:
        if c.strip().lower() == t:
            return c
    return None

def norm_node(tok):
    # 'n46' -> '46' ; if already numeric string, keep it
    if pd.isna(tok): return ""
    s = str(tok).strip()
    if s.lower().startswith('n'):
        return s[1:].strip()
    return s

# === 1) 읽기(모든 시트) ===
sheets = pd.read_excel(INPUT_XLSX, sheet_name=None, dtype=str, engine='openpyxl')

# === 2) 시트 찾기 ===
demand_sheet = find_sheet_with_columns(sheets, {'route','origin','destination'})
node_sheet = find_sheet_with_columns(sheets, {'rlnode_nm','node_id','board','alight'})

if demand_sheet is None or node_sheet is None:
    raise RuntimeError("DEMAND 시트 또는 NODE 시트를 찾을 수 없음. 컬럼 이름을 확인하세요.")

df_demand = sheets[demand_sheet].copy()
df_node = sheets[node_sheet].copy()

# 컬럼 실제 이름 찾기
route_col = find_col(df_demand, 'route')
origin_col = find_col(df_demand, 'origin')
dest_col = find_col(df_demand, 'destination')

node_id_col = find_col(df_node, 'node_id')
board_col = find_col(df_node, 'board')
alight_col = find_col(df_node, 'alight')

# 숫자형으로 변환(가능한 경우)
df_node[node_id_col] = df_node[node_id_col].astype(str).str.strip()
df_node[board_col] = pd.to_numeric(df_node[board_col], errors='coerce').fillna(0.0)
df_node[alight_col] = pd.to_numeric(df_node[alight_col], errors='coerce').fillna(0.0)

# node dicts
board_by_node = { str(r[node_id_col]).strip(): float(r[board_col]) for _, r in df_node.iterrows() }
alight_by_node = { str(r[node_id_col]).strip(): float(r[alight_col]) for _, r in df_node.iterrows() }

# === 옵션: station_line_share 매핑 (node_id -> fraction for this route)
# 만약 제공 가능하면 route별로 (예: route identifier(또는 Line))에 대해 station별 분배비를 주면 더 정확함.
# 기본: 모든 board/alight를 해당 route에 전부 사용 (즉 fraction=1.0)
# 사용자가 제공하면 아래 dict 형태: station_share = {'46':0.3, '35':0.5, ...}
# 예: station_share = {}  # 비워두면 1.0으로 처리
station_share = {}  # 사용자가 채워 넣으면 됨

def get_station_share(node_id):
    # 반환: 0..1
    return float(station_share.get(str(node_id).strip(), 1.0))

# === 방법 A: 비례할당 함수 ===
def journeys_proportional(route_tokens, origin_node, dest_node):
    # route_tokens: ['n46','n35','n31',...]
    nodes_after_origin = []
    found_origin = False
    for tok in route_tokens:
        nid = norm_node(tok)
        if not found_origin:
            if nid == origin_node:
                found_origin = True
            continue
        # origin found previously -> collect downstream nodes including first after origin
        nodes_after_origin.append(nid)
    # if destination not in downstream, return NaN
    if dest_node not in nodes_after_origin or not nodes_after_origin:
        return np.nan
    # sum of alight over downstream nodes, with optional station_share
    sum_alights = sum(alight_by_node.get(nid, 0.0) * get_station_share(nid) for nid in nodes_after_origin)
    if sum_alights == 0:
        return np.nan
    board_origin = board_by_node.get(origin_node, 0.0) * get_station_share(origin_node)
    dest_alight = alight_by_node.get(dest_node, 0.0) * get_station_share(dest_node)
    return board_origin * (dest_alight / sum_alights)

# === 방법 B: 경로 흐름 시뮬레이션 함수 ===
def simulate_route_od(route_tokens):
    """
    route_tokens: ['n46','n35','n31', ...] (order of stops along the route)
    returns: od_counts dict keyed by ('46','35') -> float journeys
    """
    # normalize tokens to node ids
    nodes = [norm_node(tok) for tok in route_tokens]
    # active pools: list of (origin_node, remaining_count)
    active = []  # list of dicts {'origin':id, 'rem':float}
    od_counts = {}  # (origin, dest) -> float

    for idx, nid in enumerate(nodes):
        # 1) boarding at this station (scaled by station_share)
        b = board_by_node.get(nid, 0.0) * get_station_share(nid)
        if b > 0:
            active.append({'origin': nid, 'rem': float(b)})

        # 2) alighting at this station
        al = alight_by_node.get(nid, 0.0) * get_station_share(nid)
        if al <= 0 or not active:
            continue

        total_onboard = sum(a['rem'] for a in active)
        # guard
        if total_onboard <= 0:
            continue

        # allocate alighting proportionally to each active origin pool
        for a in active:
            if a['rem'] <= 0:
                continue
            share = a['rem'] / total_onboard
            al_from_origin = share * al
            # decrement
            a['rem'] -= al_from_origin
            key = (a['origin'], nid)
            od_counts[key] = od_counts.get(key, 0.0) + al_from_origin

        # drop any active pools that are now near-zero
        active = [a for a in active if a['rem'] > 1e-9]

    return od_counts

# === 3) DEMAND 표에 두 방법 적용 ===
journeys_prop = []
journeys_flow = []
missing_info_prop = []
missing_info_flow = []

for _, row in df_demand.iterrows():
    route = row.get(route_col, "")
    if pd.isna(route) or str(route).strip() == "":
        journeys_prop.append(np.nan)
        journeys_flow.append(np.nan)
        missing_info_prop.append("no route")
        missing_info_flow.append("no route")
        continue

    tokens = [t.strip() for t in str(route).split('-') if t.strip() != ""]
    origin_tok = norm_node(row.get(origin_col, ""))
    dest_tok = norm_node(row.get(dest_col, ""))

    # --- proportional ---
    try:
        j_prop = journeys_proportional(tokens, origin_tok, dest_tok)
        if pd.isna(j_prop):
            missing_info_prop.append("insufficient alight/board or dest not downstream")
        else:
            missing_info_prop.append("")
    except Exception as e:
        j_prop = np.nan
        missing_info_prop.append(str(e))
    journeys_prop.append(j_prop)

    # --- flow simulation ---
    try:
        od_counts = simulate_route_od(tokens)
        j_flow = od_counts.get((origin_tok, dest_tok), 0.0)
        # if result is 0 but data suggests missing nodes, mark note
        if j_flow == 0.0:
            # detect if origin or dest absent in nodes
            if origin_tok not in [norm_node(t) for t in tokens] or dest_tok not in [norm_node(t) for t in tokens]:
                missing_info_flow.append("origin/dest missing in route")
            else:
                # zero might be truly zero (alight zero etc.)
                missing_info_flow.append("")
        else:
            missing_info_flow.append("")
    except Exception as e:
        j_flow = np.nan
        missing_info_flow.append(str(e))
    journeys_flow.append(j_flow)

# attach to df_demand
df_demand['journeys_prop'] = journeys_prop
df_demand['journeys_flow'] = journeys_flow
df_demand['note_prop'] = missing_info_prop
df_demand['note_flow'] = missing_info_flow

# write back (preserve other sheets)
sheets_out = sheets.copy()
sheets_out[demand_sheet] = df_demand

with pd.ExcelWriter(OUTPUT_XLSX, engine='openpyxl', mode='w') as writer:
    for name, df in sheets_out.items():
        sheetname = name if len(name) <= 31 else name[:31]
        df.to_excel(writer, sheet_name=sheetname, index=False)

print("완료. 결과 저장:", OUTPUT_XLSX)


완료. 결과 저장: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\qgis_export_uk_with_journeys.xlsx


#### Journeys data 추가

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# ────── CONFIG ────────────────────────────────────────────────────────────
file_path   = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx")
output_path = file_path.with_stem(file_path.stem + "_with_journeys")

min_board   = 1      # 승차가 0/누락인 역에 부여할 최소 승차 인원
min_alight  = 1      # 하차가 0/누락인 역에 부여할 최소 하차 인원
epsilon_od  = 1      # 유효 OD(승·하차>0)에 심을 최소 통행량
max_iter    = 50
eps_conv    = 1e-6
# ──────────────────────────────────────────────────────────────────────────

# 1. 데이터 읽기 -----------------------------------------------------------
bal    = pd.read_excel(
            file_path, sheet_name="BOARD_ALIGHT",
            usecols=["node_id", "board_1d", "alight_1d"]
        )
demand = pd.read_excel(file_path, sheet_name="DEMAND")

# 1‑a. Origin / Destination ⇒ 정수 node_id
demand["O_id"] = demand["Origin"].str.lstrip("n").astype(int)
demand["D_id"] = demand["Destination"].str.lstrip("n").astype(int)

# 2. BOARD_ALIGHT 누락/중복 처리 ------------------------------------------
#   2‑a. 중복 node_id 합산
bal_agg = (bal.groupby("node_id", as_index=False)
               .agg(board=("board_1d", "sum"),
                    alight=("alight_1d", "sum")))

#   2‑b. DEMAND 에 등장하지만 bal 에 없는 노드 추가
all_nodes = pd.unique(demand[["O_id", "D_id"]].values.ravel())
bal_full  = pd.DataFrame({"node_id": all_nodes}).merge(
                bal_agg, on="node_id", how="left"
            )

#   2‑c. 승·하차 0/NaN → 최소값 주입
bal_full["board"]  = pd.to_numeric(bal_full["board"],  errors="coerce").fillna(0)
bal_full["alight"] = pd.to_numeric(bal_full["alight"], errors="coerce").fillna(0)

bal_full.loc[bal_full["board"]  == 0, "board"]  = min_board
bal_full.loc[bal_full["alight"] == 0, "alight"] = min_alight

# 3. 총 승·하차량 매핑 ------------------------------------------------------
board_map  = bal_full.set_index("node_id")["board"].to_dict()
alight_map = bal_full.set_index("node_id")["alight"].to_dict()

demand["o_total"] = demand["O_id"].map(board_map)
demand["d_total"] = demand["D_id"].map(alight_map)

# 4. 초기 OD 행렬 + ε 주입 ---------------------------------------------------
f = 1.0 / demand["Timestep"].replace(0, np.nan)
demand["T"] = (demand["o_total"] * demand["d_total"] * f).fillna(0.0)

valid_od = (demand["o_total"] > 0) & (demand["d_total"] > 0)
demand.loc[valid_od, "T"] += epsilon_od      # 최소 통행량 심기

# 5. IPF (Iterative Proportional Fitting) ----------------------------------
for _ in range(max_iter):
    row_sum = demand.groupby("O_id")["T"].transform("sum").replace(0, np.nan)
    demand["T"] *= demand["o_total"] / row_sum

    col_sum = demand.groupby("D_id")["T"].transform("sum").replace(0, np.nan)
    demand["T"] *= demand["d_total"] / col_sum

    if max((row_sum - demand["o_total"]).abs().max(),
           (col_sum - demand["d_total"]).abs().max()) < eps_conv:
        break

# 6. journeys 계산 및 후처리 -------------------------------------------------
demand["journeys"] = demand["T"].round(0).astype(int)   # 정수(명)로 반올림
out = demand.drop(columns=["O_id", "D_id", "o_total", "d_total", "T"])

# 7. 결과 저장 --------------------------------------------------------------
if output_path.exists():
    mode, sheet_opt = "a", "overlay"
else:
    mode, sheet_opt = "w", None                 # 새 파일 — 시트 존재 안 하므로 옵션 불필요

with pd.ExcelWriter(output_path,
                    engine="openpyxl",
                    mode=mode,
                    if_sheet_exists=sheet_opt) as wr:
    out.to_excel(wr, sheet_name="DEMAND", index=False)

print("✅ journeys 열이 채워진 파일:", output_path)


ValueError: Worksheet named 'BOARD_ALIGHT' not found

In [48]:
import pandas as pd
from pathlib import Path

# 파일 경로
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx")
output_path = file_path.with_stem(file_path.stem + "_journeys_fixed")

# DEMAND 시트 읽기
demand = pd.read_excel(file_path, sheet_name="DEMAND")

# journeys 열에서 0인 값을 1로 변경
demand["journeys"] = demand["journeys"].apply(lambda x: 1 if x == 0 else x)

# 새 파일로 저장
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    demand.to_excel(writer, sheet_name="DEMAND", index=False)

print(f"✅ Modified file saved to: {output_path}")


✅ Modified file saved to: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export_journeys_fixed.xlsx


In [50]:
import pandas as pd
from collections import defaultdict
from pathlib import Path

# ────────── 1. 입력 경로 및 시트 읽기 ──────────
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\qgis_export_uk.xlsx")
df = pd.read_excel(file_path, sheet_name="DEMAND",
                   usecols=["Line", "Route", "journeys30"])

# ────────── 2. Edge‑별 탑승 인원 누적 ──────────
edge_load = defaultdict(int)           # {(Line, from, to): 총탑승}

for line, route, pax in df.itertuples(index=False):
    nodes = route.split("-")           # ex) ["n4","n3","n1",...]
    for frm, to in zip(nodes[:-1], nodes[1:]):
        edge_load[(line, frm, to)] += pax   # 방향 구분 O (필요 없으면 정렬해서 key 통일)

# DataFrame 화
edge_df = (pd.DataFrame([(l,f,t,c) for (l,f,t),c in edge_load.items()],
                        columns=["Line","From","To","Load"]))

# ────────── 3. 노선별 최대 탑승 인원 계산 ──────────
max_by_line = (edge_df.groupby("Line")["Load"]
                        .agg(MaxLoad="max")
                        .reset_index())

# (선택) 어느 Edge 에서 최대가 나왔는지도 보고 싶다면:
idx = edge_df.groupby("Line")["Load"].idxmax()
edge_peak = edge_df.loc[idx].reset_index(drop=True)   # Line, From, To, Load

# ────────── 4. 결과 저장 or 출력 ──────────
print("\n===== 노선별 최대 탑승 인원 =====")
print(max_by_line.to_string(index=False))

print("\n===== 최대 탑승 Edge 상세(선택) =====")
print(edge_peak.to_string(index=False))



===== 노선별 최대 탑승 인원 =====
  Line      MaxLoad
 1AWC1 62681.455319
 1AWC2  2406.977529
 1EMR1 53575.899146
 1EMR2   957.062056
  1GA1 63004.822775
  1GA2  1607.179558
 1GWR1 43215.627992
 1GWR2  2646.727970
1LNER1 71650.412764
1LNER2  4253.587918
  1SR1 33968.707205
  1SR2  1601.238405
 1TPE1  1422.917239
 1TPE2  1747.935970
  1TW1   768.871046
  1TW2  2015.183499
 2AWC1 79518.803233
 2AWC2  1156.568444
 2EMR1 70647.169439
 2EMR2  2573.045776
 2GWR1 37607.893574
 2GWR2  1813.418767
2LNER1 72429.739161
2LNER2  4393.017267
  2SR1 22464.011084
  2SR2  1131.721169
  2TW1  1785.686930
  2TW2   968.922909
 3EMR1 53157.661529
 3EMR2   877.425438
 3GWR1 27639.195702
 3GWR2   701.036900
  3SR1 46573.040225
  3SR2  1023.326345
 4EMR1 57122.331480
 4EMR2  1393.145636

===== 최대 탑승 Edge 상세(선택) =====
  Line From  To         Load
 1AWC1  n46 n35 62681.455319
 1AWC2  n15 n20  2406.977529
 1EMR1  n46 n27 53575.899146
 1EMR2  n22 n27   957.062056
  1GA1  n46 n39 63004.822775
  1GA2  n33 n36  1607.179558


In [52]:
# -*- coding: utf-8 -*-
"""
노선별 최대탑승값(주어진 리스트) + capacity_map -> 필요한 편성 수 계산
그리고 demand.json이 있으면 노선별로 편성 수만큼 균등분할하여 demand_split.json으로 저장 (옵션).
"""

import re
import math
import json
from pathlib import Path
from collections import defaultdict

# ---------------- user inputs ----------------
# capacity_map: 사용자가 제공한 값(수정 금지)
capacity_map = {
    "AWC": 607,   # Avanti West Coast (Class 390 Pendolino 최대형 기준 예시)
    "EMR": 301,   # East Midlands Railway (Class 810 등 예시)
    "GA" : 704,   # Greater Anglia (Class 745 12-car 예시)
    "GWR": 650,   # Great Western Railway (Class 800/802 대형 편성 예시)
    "LNER":600,   # LNER (Azuma 대형 편성 예시)
    "SR": 330,    # Southern (Class 377 등 4~5-car 기준 예시)
    "TPE":647,    # TransPennine Express (대형 편성 예시)
    "TW": 200,    # Transport for Wales (TfW, 2~3-car 예시)
    # 필요하면 여기에 추가 매핑
}
DEFAULT_CAPACITY = 600   # 매핑 없을 때 보수적 기본값

# 주어진 노선별 최대탑승값 (사용자가 제공한 리스트를 그대로 dict로 만듦)
# 문자열 키는 앞/뒤 숫자를 포함한 원본 노선명(예: '1AWC1')
max_loads_raw = {
"1AWC1": 62681.455319,
"1AWC2": 2406.977529,
"1EMR1": 53575.899146,
"1EMR2": 957.062056,
"1GA1": 63004.822775,
"1GA2": 1607.179558,
"1GWR1": 43215.627992,
"1GWR2": 2646.727970,
"1LNER1": 71650.412764,
"1LNER2": 4253.587918,
"1SR1": 33968.707205,
"1SR2": 1601.238405,
"1TPE1": 1422.917239,
"1TPE2": 1747.935970,
"1TW1": 768.871046,
"1TW2": 2015.183499,
"2AWC1": 79518.803233,
"2AWC2": 1156.568444,
"2EMR1": 70647.169439,
"2EMR2": 2573.045776,
"2GWR1": 37607.893574,
"2GWR2": 1813.418767,
"2LNER1": 72429.739161,
"2LNER2": 4393.017267,
"2SR1": 22464.011084,
"2SR2": 1131.721169,
"2TW1": 1785.686930,
"2TW2": 968.922909,
"3EMR1": 53157.661529,
"3EMR2": 877.425438,
"3GWR1": 27639.195702,
"3GWR2": 701.036900,
"3SR1": 46573.040225,
"3SR2": 1023.326345,
"4EMR1": 57122.331480,
"4EMR2": 1393.145636,
}

# 파일 경로 (demand.json 불러와 분할하려면 경로를 올바르게 설정)
JSON_IN  = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand.json")
JSON_OUT = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand_split_by_capacity.json")

# ---------------- helpers ----------------
def extract_letters(s: str) -> str:
    """'1AWC1' -> 'AWC' 처럼 연속된 영문자 그룹 중 첫번째를 대문자로 반환"""
    if s is None:
        return ""
    s = str(s)
    m = re.search(r"[A-Za-z]+", s)
    return m.group(0).upper() if m else ""

# ---------------- main 계산 ----------------
results = []
train_count_map = {}   # key: original line string -> n_trains
capacity_used_map = {} # key: original line -> capacity used

for line_key, maxload in max_loads_raw.items():
    op = extract_letters(line_key)          # e.g., '1AWC1' -> 'AWC'
    if not op:
        cap = DEFAULT_CAPACITY
        note = "no_letters_found -> default cap used"
    else:
        cap = capacity_map.get(op, DEFAULT_CAPACITY)
        note = "" if op in capacity_map else f"operator '{op}' not in capacity_map -> default cap used"
    # 필요한 편성 수 (올림), 최소 1
    n_trains = max(1, math.ceil(float(maxload) / float(cap))) if cap > 0 else 1
    results.append((line_key, op, float(maxload), int(cap), int(n_trains), note))
    train_count_map[line_key] = int(n_trains)
    capacity_used_map[line_key] = int(cap)

# 출력(정렬해서 보기 좋게)
print("\n=== 필요 편성 수 요약 ===")
print(f"{'Line':12s} {'Op':6s} {'MaxLoad':>12s} {'Cap':>6s} {'#Trains':>8s} {'Note'}")
for t in sorted(results, key=lambda x: x[0]):
    line_key, op, maxload, cap, n_trains, note = t
    print(f"{line_key:12s} {op:6s} {maxload:12.2f} {cap:6d} {n_trains:8d} {note}")

# ---------------- optional: demand.json 분할 ----------------
split_js = {}
if JSON_IN.exists():
    with open(JSON_IN, "r", encoding="utf-8") as f:
        demand_js = json.load(f)
    for ln, od_list in demand_js.items():
        # 사용자가 원하면 ln 자체(예: '1AWC1')를 키로 사용
        n_train = train_count_map.get(ln, 1)  # 매핑 없으면 1대
        # create keys
        for idx in range(n_train):
            split_js[f"{ln}_{idx+1}"] = []
        # distribute OD trips evenly (rounded to 2 decimals)
        for od in od_list:
            # od expected format: [o, d, j]
            if len(od) < 3:
                continue
            o, d, j = od[0], od[1], float(od[2])
            share = round(j / n_train, 2)
            for idx in range(n_train):
                split_js[f"{ln}_{idx+1}"].append([o, d, share])

    # 저장
    JSON_OUT.parent.mkdir(parents=True, exist_ok=True)
    with open(JSON_OUT, "w", encoding="utf-8") as f:
        f.write("{\n")
        for i, (line, trips) in enumerate(split_js.items()):
            line_str = json.dumps(line, ensure_ascii=False)
            trips_str = json.dumps(trips, ensure_ascii=False, separators=(",", ":"))
            comma = "," if i < len(split_js) - 1 else ""
            f.write(f"  {line_str}: {trips_str}{comma}\n")
        f.write("}\n")
    print(f"\n✅ demand split JSON 저장 완료 → {JSON_OUT}")
else:
    print(f"\n주의: {JSON_IN} 파일이 없어서 demand 분할은 수행하지 않았습니다.")

# ---------------- 끝 ----------------



=== 필요 편성 수 요약 ===
Line         Op          MaxLoad    Cap  #Trains Note
1AWC1        AWC        62681.46    607      104 
1AWC2        AWC         2406.98    607        4 
1EMR1        EMR        53575.90    301      178 
1EMR2        EMR          957.06    301        4 
1GA1         GA         63004.82    704       90 
1GA2         GA          1607.18    704        3 
1GWR1        GWR        43215.63    650       67 
1GWR2        GWR         2646.73    650        5 
1LNER1       LNER       71650.41    600      120 
1LNER2       LNER        4253.59    600        8 
1SR1         SR         33968.71    330      103 
1SR2         SR          1601.24    330        5 
1TPE1        TPE         1422.92    647        3 
1TPE2        TPE         1747.94    647        3 
1TW1         TW           768.87    200        4 
1TW2         TW          2015.18    200       11 
2AWC1        AWC        79518.80    607      132 
2AWC2        AWC         1156.57    607        2 
2EMR1        EMR        70

In [1]:
# -*- coding: utf-8 -*-
import json
from pathlib import Path

# 0) 저장 경로
OUT_DIR = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json")
OUT_PATH = OUT_DIR / "dep_time_uk.json"

# 1) 라인별 배차간격(분)
headway = {
  "1AWC1": 3, "1AWC2": 18, "1EMR1": 3, "1EMR2": 18, "1GA1": 3, "1GA2": 18,
  "1GWR1": 3, "1GWR2": 9, "1LNER1": 3, "1LNER2": 9, "1SR1": 3, "1SR2": 9,
  "1TPE1": 18, "1TPE2": 18, "1TW1": 18, "1TW2": 9, "2AWC1":3, "2AWC2": 18,
  "2EMR1": 3, "2EMR2": 9, "2GWR1": 3, "2GWR2": 18, "2LNER1": 3, "2LNER2": 9,
  "2SR1": 3, "2SR2": 18, "2TW1": 9, "2TW2": 9, "3EMR1": 3, "3EMR2": 18,
  "3GWR1": 9, "3GWR2": 18, "3SR1": 3, "3SR2": 18, "4EMR1": 3, "4EMR2": 9
}

# 2) 라인별 총 소요시간(분)
trip_time = {
  "1AWC1":31, "1AWC2":31, "1EMR1":22, "1EMR2":22, "1GA1":38, "1GA2":38,
  "1GWR1":31, "1GWR2":31, "1LNER1":59, "1LNER2":59, "1SR1":54, "1SR2":54,
  "1TPE1":31, "1TPE2":31, "1TW1":34, "1TW2":34, "2AWC1":52, "2AWC2":52,
  "2EMR1":35, "2EMR2":35, "2GWR1":46, "2GWR2":46, "2LNER1":79, "2LNER2":79,
  "2SR1":25, "2SR2":25, "2TW1":43, "2TW2":43, "3EMR1":29, "3EMR2":29,
  "3GWR1":35, "3GWR2":35, "3SR1":25, "3SR2":25, "4EMR1":34, "4EMR2":34
}

# 3) 총 시뮬레이션 길이(분 단위 timestep)
TOTAL = 72

# 4) 출발시각 생성 (도착 제한: t + trip_time[line] <= TOTAL)
dep_map = {}
for line in sorted(headway.keys()):
    h = int(headway[line])
    tt = int(trip_time.get(line, 0))   # trip_time에 없으면 0으로 간주(=제약 없음)
    if h <= 0:
        continue

    # 출발 가능한 마지막 시각(포함): t <= TOTAL - tt
    last_start = TOTAL - tt
    if last_start < 0:
        # 이동시간이 TOTAL보다 길면 해당 라인은 출발 없음
        continue

    # 0, h, 2h, ... <= last_start
    times = list(range(0, last_start + 1, h))
    for i, t in enumerate(times, start=1):
        dep_map[f"{line}_{i}"] = t

# 5) 저장
OUT_DIR.mkdir(parents=True, exist_ok=True)
with OUT_PATH.open("w", encoding="utf-8") as f:
    json.dump(dep_map, f, ensure_ascii=False, indent=2, separators=(",", ": "))

print(f"✅ 저장 완료: {OUT_PATH}")
print(f"총 출발 이벤트 수: {len(dep_map):,}")
# 샘플 몇 개 확인
for k in list(dep_map.keys())[:10]:
    print(k, ":", dep_map[k])


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\dep_time_uk.json
총 출발 이벤트 수: 229
1AWC1_1 : 0
1AWC1_2 : 3
1AWC1_3 : 6
1AWC1_4 : 9
1AWC1_5 : 12
1AWC1_6 : 15
1AWC1_7 : 18
1AWC1_8 : 21
1AWC1_9 : 24
1AWC1_10 : 27


In [4]:
# -*- coding: utf-8 -*-
"""
TRAIN 시트(RLWAY_NM, NODE) + dep_time_uk.json -> routes_nodes_uk.json
- NODE: "1-2-3-4"  -> ["n1","n2","n3","n4"] 로 변환
- dep_time_uk의 각 키(예: "1ABC1_2")에 대해 베이스("1ABC1")를 뽑아 해당 노드열을 매핑
"""

import json
from pathlib import Path
import pandas as pd
import re

# ── 경로 설정 ─────────────────────────────────────────
BASE_DIR = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json")
XLSX_PATH = BASE_DIR / "qgis_export_uk.xlsx"
DEP_JSON  = BASE_DIR / "dep_time_uk.json"
OUT_JSON  = BASE_DIR / "routes_nodes_uk.json"
TRAIN_SHEET = "TRAIN"   # 시트명이 다르면 수정

# ── 유틸 ─────────────────────────────────────────────
def to_nodes_list(node_str: str):
    """
    "46-35-31" -> ["n46","n35","n31"]
    숫자/공백/소수표기 등 유연 처리
    """
    if node_str is None:
        return []
    parts = str(node_str).strip().split("-")
    nodes = []
    for p in parts:
        p = str(p).strip()
        if p == "":
            continue
        # "46.0" 같은 경우 정수 문자열로
        try:
            if re.match(r"^\d+(\.0+)?$", p):
                p_int = int(float(p))
                nodes.append(f"n{p_int}")
            else:
                # 숫자 아니면 그대로 붙이되 접두 'n'만
                # (필요시 더 엄격히 필터링)
                nodes.append(f"n{p}")
        except Exception:
            nodes.append(f"n{p}")
    return nodes

def base_name(dep_key: str) -> str:
    """'1EMR1_3' -> '1EMR1'"""
    return dep_key.rsplit("_", 1)[0] if "_" in dep_key else dep_key

# ── 입력 로드 ────────────────────────────────────────
if not XLSX_PATH.exists():
    raise FileNotFoundError(f"엑셀 파일이 없습니다: {XLSX_PATH}")
if not DEP_JSON.exists():
    raise FileNotFoundError(f"dep_time_uk.json이 없습니다: {DEP_JSON}")

# TRAIN 시트 읽기
train_df = pd.read_excel(XLSX_PATH, sheet_name=TRAIN_SHEET, dtype=str, engine="openpyxl")
train_df.columns = [c.strip() for c in train_df.columns]

need_cols = {"RLWAY_NM", "NODE"}
missing = need_cols - set(train_df.columns)
if missing:
    raise RuntimeError(f"TRAIN 시트에 필요한 컬럼이 없습니다: {missing}")

# RLWAY_NM -> ["n..","n..",...] 매핑
map_line_to_nodes = {}
for line, nodes in train_df[["RLWAY_NM","NODE"]].itertuples(index=False):
    if line is None:
        continue
    line_key = str(line).strip()
    nodes_list = to_nodes_list(nodes)
    if nodes_list:
        map_line_to_nodes[line_key] = nodes_list

# dep_time_uk.json 읽기
with DEP_JSON.open("r", encoding="utf-8") as f:
    dep_map = json.load(f)  # {"1AWC1_1": 0, ...}

# ── 변환: dep 키마다 노드 시퀀스 부여 ────────────────
routes_nodes = {}
missing_bases = set()

# dep_map의 키 순서를 그대로 따르도록 반복
for dep_key in dep_map.keys():
    b = base_name(dep_key)
    nodes = map_line_to_nodes.get(b)
    if nodes is None:
        missing_bases.add(b)
        continue
    # 얕은 복사(리스트 보호)
    routes_nodes[dep_key] = list(nodes)

# ── 저장 ────────────────────────────────────────────
OUT_JSON.parent.mkdir(parents=True, exist_ok=True)
with OUT_JSON.open("w", encoding="utf-8") as f:
    json.dump(routes_nodes, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: {OUT_JSON}")
print(f"총 생성 항목 수: {len(routes_nodes):,}")
if missing_bases:
    print("⚠ 다음 베이스 라인은 TRAIN 시트에서 찾지 못해 스킵했습니다:")
    for b in sorted(missing_bases):
        print("  -", b)


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\routes_nodes_uk.json
총 생성 항목 수: 229


In [73]:
# -*- coding: utf-8 -*-
import json
import re
from pathlib import Path
from collections import defaultdict

BASE = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json")
DEP_PATH    = BASE / "dep_time_uk.json"   # {"1AWC1_1": 0, "1AWC1_2": 3, ...}
DEMAND_PATH = BASE / "demand_uk.json"     # {"1AWC1": [[o,d,val], ...], ...}
OUT_PATH    = BASE / "demand_uk_2.json"   # 결과 저장

# ---- 입력 로드 ----
if not DEP_PATH.exists():
    raise FileNotFoundError(f"dep_time_uk.json이 없습니다: {DEP_PATH}")
if not DEMAND_PATH.exists():
    raise FileNotFoundError(f"demand_uk.json이 없습니다: {DEMAND_PATH}")

with DEP_PATH.open("r", encoding="utf-8") as f:
    dep_map = json.load(f)
with DEMAND_PATH.open("r", encoding="utf-8") as f:
    demand_map = json.load(f)

# ---- dep 키를 (베이스, 번호)로 분해 후 정렬 ----
# 예: "1AWC1_3" -> base="1AWC1", idx=3
def split_base_idx(k: str):
    if "_" not in k:
        return k, None
    base, idx = k.rsplit("_", 1)
    # 숫자 추출 실패 시 None
    try:
        idx_num = int(idx)
    except Exception:
        idx_num = None
    return base, idx_num

# 베이스별로 키 모으고, 번호로 정렬
grouped = defaultdict(list)
for k in dep_map.keys():
    base, idx = split_base_idx(k)
    grouped[base].append((k, idx))

for base in grouped:
    grouped[base].sort(key=lambda x: (x[1] is None, x[1]))  # 숫자 있는 것 우선, 숫자 오름차순

# ---- 결과 out 생성: 각 dep 키마다 demand의 base OD 복사 ----
out = {}
missing_bases = []
for base, items in sorted(grouped.items(), key=lambda x: x[0]):
    od_list = demand_map.get(base)
    if od_list is None:
        missing_bases.append(base)
        continue
    for k, _ in items:
        # 얕은 복사로 충분 (원소는 리스트 of [o,d,val])
        out[k] = [list(od) for od in od_list]

# ---- 저장: 키마다 한 줄(엔터) ----
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
all_keys_ordered = []
for base, items in sorted(grouped.items(), key=lambda x: x[0]):
    if base in missing_bases:
        continue
    for k, _ in items:
        if k in out:
            all_keys_ordered.append(k)

with OUT_PATH.open("w", encoding="utf-8") as f:
    f.write("{\n")
    for i, k in enumerate(all_keys_ordered):
        k_json = json.dumps(k, ensure_ascii=False)
        v_json = json.dumps(out[k], ensure_ascii=False, separators=(",", ":"))
        comma = "," if i < len(all_keys_ordered) - 1 else ""
        # 키마다 한 줄씩 기록(숫자 다른 것끼리는 자연스럽게 엔터)
        f.write(f"  {k_json}: {v_json}{comma}\n")
    f.write("}\n")

print(f"✅ 저장 완료: {OUT_PATH}")
print(f"총 생성 키 수: {len(all_keys_ordered):,}")
if missing_bases:
    print("⚠ 다음 베이스 라인은 demand_uk.json에 없어 스킵되었습니다:")
    for b in sorted(missing_bases):
        print("  -", b)


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\demand_uk_2.json
총 생성 키 수: 270


In [74]:
# -*- coding: utf-8 -*-
"""
TRAIN 시트(RLWAY_NM, NODE) + dep_time_uk.json -> routes_nodes_uk.json
- NODE: "1-2-3-4"  -> ["n1","n2","n3","n4"] 로 변환
- dep_time_uk의 각 키(예: "1ABC1_2")에 대해 베이스("1ABC1")를 뽑아 해당 노드열을 매핑
"""

import json
from pathlib import Path
import pandas as pd
import re

# ── 경로 설정 ─────────────────────────────────────────
BASE_DIR = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json")
XLSX_PATH = BASE_DIR / "qgis_export_uk.xlsx"
DEP_JSON  = BASE_DIR / "dep_time_uk.json"
OUT_JSON  = BASE_DIR / "routes_nodes_uk.json"
TRAIN_SHEET = "TRAIN"   # 시트명이 다르면 수정

# ── 유틸 ─────────────────────────────────────────────
def to_nodes_list(node_str: str):
    """
    "46-35-31" -> ["n46","n35","n31"]
    숫자/공백/소수표기 등 유연 처리
    """
    if node_str is None:
        return []
    parts = str(node_str).strip().split("-")
    nodes = []
    for p in parts:
        p = str(p).strip()
        if p == "":
            continue
        # "46.0" 같은 경우 정수 문자열로
        try:
            if re.match(r"^\d+(\.0+)?$", p):
                p_int = int(float(p))
                nodes.append(f"n{p_int}")
            else:
                # 숫자 아니면 그대로 붙이되 접두 'n'만
                # (필요시 더 엄격히 필터링)
                nodes.append(f"n{p}")
        except Exception:
            nodes.append(f"n{p}")
    return nodes

def base_name(dep_key: str) -> str:
    """'1EMR1_3' -> '1EMR1'"""
    return dep_key.rsplit("_", 1)[0] if "_" in dep_key else dep_key

# ── 입력 로드 ────────────────────────────────────────
if not XLSX_PATH.exists():
    raise FileNotFoundError(f"엑셀 파일이 없습니다: {XLSX_PATH}")
if not DEP_JSON.exists():
    raise FileNotFoundError(f"dep_time_uk.json이 없습니다: {DEP_JSON}")

# TRAIN 시트 읽기
train_df = pd.read_excel(XLSX_PATH, sheet_name=TRAIN_SHEET, dtype=str, engine="openpyxl")
train_df.columns = [c.strip() for c in train_df.columns]

need_cols = {"RLWAY_NM", "NODE"}
missing = need_cols - set(train_df.columns)
if missing:
    raise RuntimeError(f"TRAIN 시트에 필요한 컬럼이 없습니다: {missing}")

# RLWAY_NM -> ["n..","n..",...] 매핑
map_line_to_nodes = {}
for line, nodes in train_df[["RLWAY_NM","NODE"]].itertuples(index=False):
    if line is None:
        continue
    line_key = str(line).strip()
    nodes_list = to_nodes_list(nodes)
    if nodes_list:
        map_line_to_nodes[line_key] = nodes_list

# dep_time_uk.json 읽기
with DEP_JSON.open("r", encoding="utf-8") as f:
    dep_map = json.load(f)  # {"1AWC1_1": 0, ...}

# ── 변환: dep 키마다 노드 시퀀스 부여 ────────────────
routes_nodes = {}
missing_bases = set()

# dep_map의 키 순서를 그대로 따르도록 반복
for dep_key in dep_map.keys():
    b = base_name(dep_key)
    nodes = map_line_to_nodes.get(b)
    if nodes is None:
        missing_bases.add(b)
        continue
    # 얕은 복사(리스트 보호)
    routes_nodes[dep_key] = list(nodes)

# ── 저장 ────────────────────────────────────────────
OUT_JSON.parent.mkdir(parents=True, exist_ok=True)
with OUT_JSON.open("w", encoding="utf-8") as f:
    json.dump(routes_nodes, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: {OUT_JSON}")
print(f"총 생성 항목 수: {len(routes_nodes):,}")
if missing_bases:
    print("⚠ 다음 베이스 라인은 TRAIN 시트에서 찾지 못해 스킵했습니다:")
    for b in sorted(missing_bases):
        print("  -", b)


✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\routes_nodes_uk.json
총 생성 항목 수: 270


In [67]:
import pandas as pd
from pathlib import Path

# 1) 파일·시트 설정
xlsx_fp = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json\qgis_export_uk.xlsx")
sheet   = "DEMAND"  # 시트명이 다르면 수정

# 2) 필요한 컬럼만 읽기
usecols = ["Line", "timestep10"]
df = pd.read_excel(xlsx_fp, sheet_name=sheet, usecols=usecols, dtype=str, engine="openpyxl")

# 3) 전처리: 공백 제거 + 수치화
df["Line"] = df["Line"].astype(str).str.strip()
df["timestep10"] = pd.to_numeric(df["timestep10"], errors="coerce")

# 4) Line별 최대 timestep10 계산
max_by_line = (df.groupby("Line", dropna=True)["timestep10"]
                 .max()
                 .reset_index()
                 .rename(columns={"timestep10": "max_timestep10"}))

# 5) 정렬(원하면 내림차순)
max_by_line = max_by_line.sort_values(["Line"]).reset_index(drop=True)

# 6) 출력
print("=== Line별 최대 timestep10 ===")
for line, vmax in max_by_line.itertuples(index=False):
    print(f"{line}\t{vmax}")

# (선택) 표 형태로 한 번에 보기
# print(max_by_line.to_string(index=False))


=== Line별 최대 timestep10 ===
1AWC1	23
1AWC2	23
1EMR1	17
1EMR2	17
1GA1	32
1GA2	32
1GWR1	23
1GWR2	23
1LNER1	49
1LNER2	49
1SR1	47
1SR2	47
1TPE1	26
1TPE2	26
1TW1	27
1TW2	27
2AWC1	43
2AWC2	43
2EMR1	28
2EMR2	28
2GWR1	37
2GWR2	37
2LNER1	67
2LNER2	67
2SR1	21
2SR2	21
2TW1	35
2TW2	35
3EMR1	23
3EMR2	23
3GWR1	29
3GWR2	29
3SR1	20
3SR2	20
4EMR1	28
4EMR2	28


In [64]:
import pandas as pd, json, math
from collections import defaultdict
from pathlib import Path

# ──────────────────────── 경로 설정 ────────────────────────
BASE     = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\8.UK\json")
xlsx_fp  = BASE / "qgis_export_uk.xlsx"
json_in  = BASE / "json" / "demand.json"
json_out = BASE / "json" / "demand_split.json"

# ───────────────── 1. DEMAND 시트 → Edge 최대 탑승 ─────────────────
df = pd.read_excel(xlsx_fp, sheet_name="DEMAND",
                   usecols=["Line", "Route", "journeys"])

edge_load = defaultdict(int)          # {(Line, u, v): 승객 누적}

for line, route, pax in df.itertuples(index=False):
    nodes = route.split("-")
    for u, v in zip(nodes[:-1], nodes[1:]):
        edge_load[(line, u, v)] += pax

edge_df = (pd.DataFrame([(l,u,v,c) for (l,u,v),c in edge_load.items()],
                        columns=["Line","From","To","Load"]))

max_load = (edge_df.groupby("Line")["Load"]
                     .max()
                     .to_dict())      # {Line: 최대 Edge 탑승}

# ───────────────── 2. 노선별 열차 1편성 최대 좌석수 매핑 ────────────────
def train_capacity(line: str) -> int:
    """노선 이름을 받아 보수적 1편성 최대 좌석수를 반환"""
    if line.startswith("KTX강릉선"):
        return 381           # KTX‑Eum
    if line.startswith("경부고속철도"):
        return 515           # KTX‑청룡 / 산천
    # 그 외 일반선·호남고속철도 등 → 산천 보수치
    return 410               # KTX‑Sancheon

cap_map   = {ln: train_capacity(ln) for ln in max_load}
train_cnt = {ln: max(1, math.ceil(max_load[ln] / cap_map[ln]))
             for ln in max_load}

print("\n=== 노선별 최대 Edge 탑승·필요 열차 대수 ===")
for ln in sorted(train_cnt):
    print(f"{ln:15s}  최대탑승 = {max_load[ln]:5.0f}명  "
          f"/  용량 = {cap_map[ln]}  →  열차 {train_cnt[ln]}대")

# ───────────────── 3. demand.json 불러와서 쪼개기 ─────────────────
with open(json_in, "r", encoding="utf-8") as f:
    demand_js = json.load(f)

split_js = {}   # 새로운 dict

for ln, od_list in demand_js.items():
    n_train = train_cnt.get(ln, 1)          # 매핑 안 되면 1대
    for idx in range(n_train):
        key = f"{ln}_{idx+1}"
        split_js[key] = []
    for o, d, j in od_list:
        share = round(j / n_train, 2)
        for idx in range(n_train):
            split_js[f"{ln}_{idx+1}"].append([o, d, share])

# ───────────────── 4. JSON 저장 (한 노선 = 한 줄) ─────────────────
with open(json_out, "w", encoding="utf-8") as f:
    f.write("{\n")
    for i, (line, trips) in enumerate(split_js.items()):
        line_str = json.dumps(line, ensure_ascii=False)
        trips_str = json.dumps(trips, ensure_ascii=False, separators=(",", ":"))
        comma = "," if i < len(split_js) - 1 else ""
        f.write(f"  {line_str}: {trips_str}{comma}\n")
    f.write("}\n")

print(f"\n✅ 줄바꿈 최소화된 JSON 저장 완료 → {json_out}")



=== 노선별 최대 Edge 탑승·필요 열차 대수 ===
1AWC1            최대탑승 = 1880444명  /  용량 = 410  →  열차 4587대
1AWC2            최대탑승 = 72209명  /  용량 = 410  →  열차 177대
1EMR1            최대탑승 = 1607277명  /  용량 = 410  →  열차 3921대
1EMR2            최대탑승 = 28712명  /  용량 = 410  →  열차 71대
1GA1             최대탑승 = 1890145명  /  용량 = 410  →  열차 4611대
1GA2             최대탑승 = 48215명  /  용량 = 410  →  열차 118대
1GWR1            최대탑승 = 1296469명  /  용량 = 410  →  열차 3163대
1GWR2            최대탑승 = 79402명  /  용량 = 410  →  열차 194대
1LNER1           최대탑승 = 2149512명  /  용량 = 410  →  열차 5243대
1LNER2           최대탑승 = 127608명  /  용량 = 410  →  열차 312대
1SR1             최대탑승 = 1019061명  /  용량 = 410  →  열차 2486대
1SR2             최대탑승 = 48037명  /  용량 = 410  →  열차 118대
1TPE1            최대탑승 = 42688명  /  용량 = 410  →  열차 105대
1TPE2            최대탑승 = 52438명  /  용량 = 410  →  열차 128대
1TW1             최대탑승 = 23066명  /  용량 = 410  →  열차 57대
1TW2             최대탑승 = 60456명  /  용량 = 410  →  열차 148대
2AWC1            최대탑승 = 2385564명  /  용량 = 410  →  열차 5

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\MINJI\\NETWORK RELIABILITY\\QGIS\\8.UK\\json\\json\\demand.json'

출발 timestep 설정

In [17]:
import pandas as pd
import math

# 파일 경로
file_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\qgis_export.xlsx"
sheet_name = "EDGE"

# 엑셀 불러오기
df = pd.read_excel(file_path, sheet_name=sheet_name)

# 'time(min)' 열이 있는지 확인
if "time(min)" not in df.columns:
    raise KeyError("'time(min)' 열이 존재하지 않습니다.")

# 소숫점 올림하여 time_step_15 계산
df["time_step_15"] = df["time(min)"].apply(lambda x: math.ceil(x / 15))

# 덮어쓰기 저장
with pd.ExcelWriter(file_path, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ 'time_step_15' 열이 성공적으로 추가되었습니다.")


✅ 'time_step_15' 열이 성공적으로 추가되었습니다.


In [ ]:
import json
import os

# 파일 경로
base_dir = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"
dep_path = os.path.join(base_dir, "dep_time.json")
demand_path = os.path.join(base_dir, "demand.json")
routes_path = os.path.join(base_dir, "routes_nodes.json")

# 1. dep_time.json 로드
with open(dep_path, encoding="utf-8") as f:
    dep_data = json.load(f)
valid_trains = set(dep_data.keys())

# 2. demand.json 필터링
with open(demand_path, encoding="utf-8") as f:
    demand_data = json.load(f)
filtered_demand = {k: v for k, v in demand_data.items() if k in valid_trains}

# 3. routes_nodes.json 필터링
with open(routes_path, encoding="utf-8") as f:
    routes_data = json.load(f)
filtered_routes = {k: v for k, v in routes_data.items() if k in valid_trains}

# 4. 저장 (덮어쓰기)
with open(demand_path, "w", encoding="utf-8") as f:
    json.dump(filtered_demand, f, ensure_ascii=False, indent=2)

with open(routes_path, "w", encoding="utf-8") as f:
    json.dump(filtered_routes, f, ensure_ascii=False, indent=2)

print("✅ demand.json과 routes_nodes.json이 dep_time.json 기준으로 성공적으로 필터링되었습니다.")


✅ dep_time.json trimmed and saved (cutoff by rounded threshold).


In [34]:
import json
import os

# 경로 설정
base_dir = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json"
demand_path = os.path.join(base_dir, "demand_filtered.json")
routes_path = os.path.join(base_dir, "routes_nodes_filtered.json")

def save_single_line_json(data, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        f.write("{\n")
        for i, (k, v) in enumerate(data.items()):
            line = f'  "{k}": {json.dumps(v, ensure_ascii=False)}'
            if i < len(data) - 1:
                line += ","
            f.write(line + "\n")
        f.write("}")

# 파일 불러오기
with open(demand_path, encoding="utf-8") as f:
    demand_data = json.load(f)
with open(routes_path, encoding="utf-8") as f:
    routes_data = json.load(f)

# 한 줄당 기차 하나씩 저장
save_single_line_json(demand_data, demand_path)
save_single_line_json(routes_data, routes_path)

print("✅ 줄바꿈 포맷 정리 완료.")


✅ 줄바꿈 포맷 정리 완료.


In [1]:
import json
from pathlib import Path

# ───────────────────────────────
# 1. 파일 경로 설정
file_path = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand.json")
output_path = file_path.parent / "demand_filtered.json"

# ───────────────────────────────
# 2. 데이터 불러오기
with open(file_path, "r", encoding="utf-8") as f:
    demand_data = json.load(f)

# ───────────────────────────────
# 3. 필터링 및 통계
original_count = 0
filtered_count = 0

filtered_data = {}

for train_id, od_list in demand_data.items():
    original_count += len(od_list)
    
    # 수요 10 이상인 OD쌍만 남기기
    new_od_list = [od for od in od_list if od[2] >= 10]
    filtered_count += len(new_od_list)
    
    # 결과가 남아있는 경우에만 저장
    if new_od_list:
        filtered_data[train_id] = new_od_list

# ───────────────────────────────
# 4. 필터링 결과 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 5. 결과 출력
print(f"🔹 필터링 전 OD쌍 개수: {original_count}")
print(f"🔹 필터링 후 OD쌍 개수: {filtered_count}")
print(f"🔹 제거된 OD쌍 개수: {original_count - filtered_count}")
print(f"✅ 저장 완료: {output_path}")


🔹 필터링 전 OD쌍 개수: 12426
🔹 필터링 후 OD쌍 개수: 3607
🔹 제거된 OD쌍 개수: 8819
✅ 저장 완료: D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json\demand_filtered.json


In [3]:
import json
from pathlib import Path

# ───────────────────────────────
# 1. 경로 설정
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\json")
dep_time_path = base_dir / "dep_time.json"
routes_nodes_path = base_dir / "routes_nodes.json"
dep_time_out = base_dir / "dep_time_filtered.json"
routes_nodes_out = base_dir / "routes_nodes_filtered.json"

# ───────────────────────────────
# 2. dep_time.json 불러오기 및 필터링 (값이 13 미만인 것만)
with open(dep_time_path, "r", encoding="utf-8") as f:
    dep_time = json.load(f)

filtered_dep_time = {k: v for k, v in dep_time.items() if v < 12}
print(f"✅ dep_time 필터링: {len(dep_time)} → {len(filtered_dep_time)}개 남음")

# 저장
with open(dep_time_out, "w", encoding="utf-8") as f:
    json.dump(filtered_dep_time, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 3. routes_nodes.json 불러오기 및 해당 키만 남기기
with open(routes_nodes_path, "r", encoding="utf-8") as f:
    routes_nodes = json.load(f)

# dep_time에서 남은 키만 유지
filtered_routes_nodes = {k: v for k, v in routes_nodes.items() if k in filtered_dep_time}
print(f"✅ routes_nodes 필터링: {len(routes_nodes)} → {len(filtered_routes_nodes)}개 남음")

# 저장
with open(routes_nodes_out, "w", encoding="utf-8") as f:
    json.dump(filtered_routes_nodes, f, ensure_ascii=False, indent=2)

# ───────────────────────────────
# 4. 결과 요약
print(f"\n📁 저장 완료:")
print(f"→ {dep_time_out.name}")
print(f"→ {routes_nodes_out.name}")


✅ dep_time 필터링: 122 → 106개 남음
✅ routes_nodes 필터링: 122 → 106개 남음

📁 저장 완료:
→ dep_time_filtered.json
→ routes_nodes_filtered.json


#### Capacity
- Demand_data이용해서 엣지마다 흐르는 demand양 계산한 파일: 6.edge_journeys_summary.xlsx
- e1 n1-n2 1502241
- e2 n2-n1 389459.5
- e1,e2,e3,e4, .. 넘버링 qgis와 다름 (양방향 고려)
- 양방향 엣지에 대해서는 서로 동일한 capacity를 갖도록 해야함

In [3]:
import pandas as pd
from pathlib import Path

# 1. 파일 불러오기
base_dir = Path(r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full")
input_path = base_dir / "6.edge_journeys_summary.xlsx"
df = pd.read_excel(input_path)

# 2. 여유 계수 설정 (예: 1.2배)
buffer_ratio = 1.2

# 3. (u, v) / (v, u) 형태를 고려해 양방향 키 만들기
df["pair_key"] = df.apply(lambda row: tuple(sorted([row["from_node_name"], row["to_node_name"]])), axis=1)

# 4. 같은 pair_key 그룹 내에서 최대 journeys 선택
grouped = df.groupby("pair_key", as_index=False).agg({
    "journeys": "max"
})
grouped["capacity"] = grouped["journeys"] * buffer_ratio

# 5. 다시 원래 방향성 edge에 capacity 연결
df = df.merge(grouped[["pair_key", "capacity"]], on="pair_key", how="left")

# 6. 컬럼 정리 및 저장
df = df.drop(columns=["pair_key"])
output_path = base_dir / "7.edge_capacity_summary.xlsx"
df.to_excel(output_path, index=False)

print(f"Capacity summary saved → {output_path}")


Capacity summary saved → D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\7.edge_capacity_summary.xlsx


#### 노선별 승객 수요 데이터 -> 필요한 기차수 계산

In [31]:
import json, math, pandas as pd, os

# ───────────────────────────────────────────────
# 1. 파일 읽기
file_path = r"D:\MINJI\NETWORK RELIABILITY\QGIS\7.Korea_Full\5.demand_data_kofull.json"
df = pd.read_json(file_path)
df["journeys_day"] = df["journeys"] / 365

# ───────────────────────────────────────────────
# 2. 파라미터
TARGET_LOAD    = 1.0          # 100 % 탑승률
TURNAROUND_MIN = 15           # 종점 버퍼(분)
HOURS_PER_DAY  = 17
DEFAULT_SEAT   = 400          # seat_map 에 없을 때 임시값

# 노선별 좌석수 (앞 글자 매칭용)
seat_map = {
    # 고속열차
    "KTX강릉선":    381,   # KTX-이음 (KTX-Eum) 381석 :contentReference[oaicite:2]{index=2}
    "KTX경부선":    955,   # KTX-I 955석(935~955) :contentReference[oaicite:3]{index=3}
    "KTX호남선":    955,
    "경부고속철도":   955,   # KTX-산천
    "경부선":         900,   # SRT 편성(예시)
    # ITX 계열
    "ITX-새마을":   376,   # ITX-새마을
    "ITX-청춘":     402,   # ITX-청춘
    # 일반열차
    "무궁화호":     920,   # 좌석+입석 허용, 편성 좌석 920 부근
}

# ───────────────────────────────────────────────
def train_stats(group):
    line = group.name
    seats = next((v for k, v in seat_map.items() if line.startswith(k)), DEFAULT_SEAT)

    total_pax = group["journeys_day"].sum()
    single_run = group["time_min"].max()
    round_trip = single_run + TURNAROUND_MIN
    cycles = max(1, (HOURS_PER_DAY * 60) // round_trip)

    cap_per_set = seats * TARGET_LOAD * cycles
    sets_needed = math.ceil(total_pax / cap_per_set) if cap_per_set else None

    return pd.Series({
        "cycles_day": cycles,
        "train_sets_needed": sets_needed
    })

# 4. 계산
result = df.groupby("line").apply(train_stats).reset_index()

print(result)


      line  cycles_day  train_sets_needed
0   KTX강릉선          24                  1
1   KTX동해선          39                  1
2      경강선          23                  2
3   경부고속철도           8                 23
4      경부선           1                 65
5      경북선          18                  1
6      경의선          46                  1
7      경인선          35                  1
8      경전선           5                  3
9     공항철도          20                  2
10     대구선          44                  1
11   동해남부선          12                  3
12     동해선          16                  1
13     삼척선          51                  1
14     영동선          12                  2
15     장항선          11                  2
16     전라선          17                  2
17   중부내륙선          36                  1
18     중앙선           2                 23
19     충북선          11                  1
20     태백선          19                  1
21  호남고속철도          20                  1
22     호남선          10            

C:\Users\Minji Kang\AppData\Local\Temp\ipykernel_38976\2930921966.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby("line").apply(train_stats).reset_index()
